In [8]:
import pandas as pd

info = pd.read_csv("studentInfo.csv")
reg = pd.read_csv("studentRegistration.csv")
vle = pd.read_csv("studentVle.csv")
assess = pd.read_csv("studentAssessment.csv")
assessments = pd.read_csv("assessments.csv")
vle_meta = pd.read_csv("vle.csv")
courses = pd.read_csv("courses.csv")


In [2]:
merged = info.merge(reg, on=["id_student", "code_module", "code_presentation"])


defining dropout labels A student dropped out if date_unregistration is not null.

In [3]:
merged["dropped_out"] = merged["date_unregistration"].notnull().astype(int)


 Calculate Time-to-Event
Create survival labels: event = 1 if dropped, time = number of weeks from start.

In [4]:
# Students who dropped: time = weeks until unregistration
merged["event"] = merged["dropped_out"]
merged["duration"] = merged["date_unregistration"].fillna(269) - merged["date_registration"]


In [5]:

# Aggregate weekly VLE clicks
vle_agg = vle.groupby(["id_student", "code_module", "code_presentation", "week_from_start"]).agg({
    "sum_click": ["sum", "mean", "std", "count"]
}).reset_index()
vle_agg.columns = ["id_student", "code_module", "code_presentation", "week", "total_clicks", "mean_clicks", "std_clicks", "active_days"]


KeyError: 'week_from_start'

In [11]:
import pandas as pd

# Load studentVle dataset
student_vle = pd.read_csv("studentVle.csv")

# Check the columns again
print(student_vle.columns)


Index(['code_module', 'code_presentation', 'id_student', 'id_site', 'date',
       'sum_click'],
      dtype='object')


create week from start


In [12]:
# Convert days to week number (0-indexed)
student_vle['week_from_start'] = (student_vle['date'] // 7).astype(int)


In [13]:
vle_agg = student_vle.groupby(
    ["id_student", "code_module", "code_presentation", "week_from_start"]
).agg(
    total_clicks=("sum_click", "sum"),
    mean_clicks=("sum_click", "mean"),
    std_clicks=("sum_click", "std"),
    active_days=("sum_click", "count")
).reset_index()


Aggregate VLE Features Per Student (Static Features for Cox Model)

In [14]:
vle_static = vle_agg.groupby(["id_student", "code_module", "code_presentation"]).agg(
    vle_total_clicks=("total_clicks", "sum"),
    vle_mean_clicks=("mean_clicks", "mean"),
    vle_std_clicks=("std_clicks", "mean"),
    vle_weeks_active=("active_days", "count")
).reset_index()


In [17]:
student_features = pd.merge(
    student_df,
    vle_static,
    on=["id_student", "code_module", "code_presentation"],
    how="left"
)



In [16]:
# Load studentInfo and studentRegistration again (if not already loaded)
student_info = pd.read_csv("studentInfo.csv")
student_reg = pd.read_csv("studentRegistration.csv")

# Merge to create student_df
student_df = pd.merge(
    student_info,
    student_reg,
    on=["id_student", "code_module", "code_presentation"],
    how="left"
)

# Define dropout flag and survival duration
student_df["dropped_out"] = student_df["date_unregistration"].notnull().astype(int)
student_df["event"] = student_df["dropped_out"]
student_df["duration"] = student_df["date_unregistration"].fillna(269) - student_df["date_registration"]


In [20]:
# Group by student-course and aggregate assessment data
assessment_scores = student_assess.groupby(
    ["id_student", "code_module", "code_presentation"]
).agg(
    mean_score=("score", "mean"),
    num_submissions=("score", "count"),
    missed_submissions=("score", lambda x: x.isna().sum())
).reset_index()


KeyError: 'code_module'

In [19]:
import pandas as pd

# Load studentAssessment file
student_assess = pd.read_csv("studentAssessment.csv")


In [21]:
print(student_assess.columns)


Index(['id_assessment', 'id_student', 'date_submitted', 'is_banked', 'score'], dtype='object')


In [22]:
assessments = pd.read_csv("assessments.csv")


In [23]:
student_assess = pd.merge(
    student_assess,
    assessments[["id_assessment", "code_module", "code_presentation"]],
    on="id_assessment",
    how="left"
)


In [24]:
assessment_scores = student_assess.groupby(
    ["id_student", "code_module", "code_presentation"]
).agg(
    mean_score=("score", "mean"),
    num_submissions=("score", "count"),
    missed_submissions=("score", lambda x: x.isna().sum())
).reset_index()


In [25]:
# Drop rows where duration is negative or null
student_features = student_features[student_features["duration"] >= 0]


In [26]:
print(student_features.shape)
student_features.describe(include='all')


(32548, 21)


,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,...,final_result,date_registration,date_unregistration,dropped_out,event,duration,vle_total_clicks,vle_mean_clicks,vle_std_clicks,vle_weeks_active
count,32548,32548,3.254800e+04,32548,32548,32548,31437,32548,32548.000000,32548.000000,...,32548,32548.000000,10033.000000,32548.000000,32548.000000,32548.000000,29221.000000,29221.000000,28929.000000,29221.000000
unique,7,4,NaN,2,13,5,10,3,NaN,NaN,...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,BBB,2014J,NaN,M,Scotland,A Level or Equivalent,20-30%,0-35,NaN,NaN,...,Pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,7900,11251,NaN,17849,3445,14026,3651,22911,NaN,NaN,...,12360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,7.061734e+05,NaN,NaN,NaN,NaN,NaN,0.163144,79.714422,...,NaN,-69.411300,50.276189,0.308252,0.308252,270.989154,1355.290750,2.845514,3.456110,21.456658
std,NaN,NaN,5.483864e+05,NaN,NaN,NaN,NaN,NaN,0.479452,41.046179,...,NaN,49.260522,82.028804,0.461779,0.461779,113.100494,1733.671146,1.022288,2.313673,12.457570
min,NaN,NaN,3.733000e+03,NaN,NaN,NaN,NaN,NaN,0.000000,30.000000,...,NaN,-322.000000,-317.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
25%,NaN,NaN,5.085320e+05,NaN,NaN,NaN,NaN,NaN,0.000000,60.000000,...,NaN,-100.000000,-1.000000,0.000000,0.000000,208.000000,261.000000,2.129827,1.896698,10.000000
50%,NaN,NaN,5.902380e+05,NaN,NaN,NaN,NaN,NaN,0.000000,60.000000,...,NaN,-57.000000,28.000000,0.000000,0.000000,299.000000,740.000000,2.666084,2.901892,23.000000
75%,NaN,NaN,6.443902e+05,NaN,NaN,NaN,NaN,NaN,0.000000,120.000000,...,NaN,-29.000000,109.000000,1.000000,1.000000,343.000000,1770.000000,3.401538,4.525334,33.000000


In [27]:
student_features.to_csv("student_features_cleaned.csv", index=False)
